In [63]:
pip install scikit-learn nltk


In [64]:
import pandas as pd
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [65]:
x=pd.read_csv("/content/COVID19_FAQ.csv")
x

,questions,answers
0,1. How does COVID-19 spread?,People can catch COVID-19 from others who have...
1,2. What are the symptoms of COVID-19?,The most common symptoms of COVID-19 are fever...
2,3. How do I know if it is COVID-19 or just the...,A COVID-19 infection has the same signs and sy...
3,4. Can the virus that causes COVID-19 be trans...,Studies to date suggest that the virus that ca...
4,5. What can I do to protect myself and prevent...,Protection measures for everyone Stay aware ...
...,...,...
62,63. Is the virus that causes COVID-19 found in...,The virus that causes COVID-19 has been found ...
63,64. Can the COVID-19 virus spread through sewe...,"Recently, ribonucleic acid (RNA) from the viru..."
64,65. Should wastewater workers take extra preca...,"Recently, ribonucleic acid (RNA) from the viru..."
65,66. If my utility has issued a Boil Water Advi...,"In most cases, it is safe to wash your hands w..."


In [66]:
df=pd.DataFrame(x)
df.head()

,questions,answers
0,1. How does COVID-19 spread?,People can catch COVID-19 from others who have...
1,2. What are the symptoms of COVID-19?,The most common symptoms of COVID-19 are fever...
2,3. How do I know if it is COVID-19 or just the...,A COVID-19 infection has the same signs and sy...
3,4. Can the virus that causes COVID-19 be trans...,Studies to date suggest that the virus that ca...
4,5. What can I do to protect myself and prevent...,Protection measures for everyone Stay aware ...


In [67]:
df.isnull().sum()

,0
questions,0
answers,0


In [68]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess(text):
    text = text.lower()
    text = re.sub(r'{.*?}', '', text)  # remove placeholders
    text = re.sub(f"[{string.punctuation}]", "", text)
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return ' '.join(tokens)


df['Cleaned_Description'] = df['questions'].apply(preprocess)


In [69]:
import re

df['Cleaned_Description1'] = df['Cleaned_Description'].str.replace(r'^(?:[1-9]|[1-5][0-9]|6[0-8])\s+', '', regex=True)


In [70]:
df['Cleaned_Description1']

,Cleaned_Description1
0,covid19 spread
1,symptom covid19
2,know covid19 common flu
3,virus cause covid19 transmitted air
4,protect prevent spread disease
...,...
62,virus cause covid19 found feces stool
63,covid19 virus spread sewerage system
64,wastewater worker take extra precaution protec...
65,utility issued boil water advisory still use t...


In [71]:
# Install required library
!pip install sentence-transformers

import pandas as pd
import re
from sentence_transformers import SentenceTransformer, util


In [72]:
# Load BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [73]:
# Generate embeddings
corpus = df['Cleaned_Description1'].tolist()
corpus_embeddings = model.encode(corpus, convert_to_tensor=True)


In [74]:
# Define chatbot function
def get_faq_response(user_query):
    query_embedding = model.encode(user_query, convert_to_tensor=True)
    similarity_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)

    best_match_idx = similarity_scores.argmax().item()
    best_score = similarity_scores[0][best_match_idx].item()

    if best_score > 0.5:
        return df.iloc[best_match_idx]['answers']
    else:
        return "I'm not sure about that. Can you rephrase your question?"

# Chat loop
print("🤖 Hello! I'm SupportBot — your virtual assistant.")
print("🤖 COVID-19 FAQ Chatbot Ready! Ask your question (type 'exit' to quit):\n")
print("💬 Ask me about your issue or support query.")

while True:
    user_input = input("You: ")
    if user_input.lower().strip() in ['exit', 'quit']:
        print("Bot: Take care! Stay safe! 😷")
        break
    response = get_faq_response(user_input)
    print("Bot:", response)

🤖 Hello! I'm SupportBot — your virtual assistant.
🤖 COVID-19 FAQ Chatbot Ready! Ask your question (type 'exit' to quit):

💬 Ask me about your issue or support query.
You: how covid19 spread
Bot: People can catch COVID-19 from others who have the virus. The disease can spread from person   to person through small droplets from the nose or mouth which are spread when a person with   COVID-19 coughs or exhales. These droplets land on objects and surfaces around the person.   Other people then catch COVID-19 by touching these objects or surfaces, then touching their   eyes, nose or mouth. People can also catch COVID-19 if they breathe in droplets from a person   with COVID-19 who coughs out or exhales droplets. This is why it is important to stay more than   1 meter (3 feet) away from a person who is sick.
You: exit
Bot: Take care! Stay safe! 😷
